In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [37]:
from __future__ import division

import gym
import numpy as np
import pandas as pd
import random
import tensorflow as tf  

import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow_addons.layers as layers
from sklearn.metrics import confusion_matrix, classification_report

from gym import spaces

import math

try:
    xrange = xrange
except:
    xrange = range

In [12]:
df = pd.read_csv('./seizure_data_no_patient.csv')

In [13]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [42]:
Reward_adjustment = 3
# Refers to Reward structure. Is imbalance Ratio of classes.
# This ratio/reward maybe tuned to improve model Recall
lamb_da = (1908/10422) * Reward_adjustment

In [43]:
lamb_da

0.5492227979274611

In [44]:
class DataFrameEnv(gym.Env):
 
    def __init__(self):
        
        #Init Action Space with to Discrete steps. 0 or 1, Left or right, On or off.
        self.action_space = spaces.Discrete(2)
        #Init Observation space 26x1 that contain continous values. change shape to fit any dataframe(in theory)
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(178,), dtype=np.float32)
        
        #Init the rest of DataFrameEnv class variables
        self.episode_df = None
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None
        self.episode = None
        self.test_set_no_labels = None
        self.test_labels = None

    def reset(self):
        # Reset function randomly samples Dataframe, Splits off labels and converts to list
        self.episode_df = df.sample(frac=0.8)
        self.episode = self.episode_df.drop(columns=['y']).values.tolist()
        self.true_labels = self.episode_df['y'].values.tolist()
        
        try:
            # Remove one row from sampled DF 
            self.state = self.episode.pop()
        except:
            self.state = None

        self.steps_beyond_done = None
        # Return to agent as a state.
        return np.array(self.state)

    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        
        # Pull off next row and assign to state to be returned to agent from this function
        state = self.episode.pop()
        

        done = False
        true_label = self.true_labels.pop()
        # Compare agents prediction with label and assign reward
        
        if true_label is not None:
            
            if (true_label == 1) & (action == true_label):
                reward = 1
            
            elif (true_label == 0) & (action == true_label):
                reward = lamb_da
            
            elif (true_label == 0) & (action != true_label):
                reward = -lamb_da
            
            else:
                reward = -1
                done = True
        else:
            reward = 0
            done = True
        # Return Next state, reward for action in current state and if done/ stop episode.
        return np.array(state), reward, done, {}

    def validate(self):
        # Validation function compare predictions to actuals on test set to build Confusion Matrix
        test_set = df[~df.index.isin(self.episode_df.index)]
        self.test_set_no_labels = test_set.drop(columns=['y'])
        self.test_labels = test_set['y']
        
        return self.test_set_no_labels, self.test_labels

In [45]:
# Init Environment
env = DataFrameEnv()

In [46]:
# Hyperparameters
H = 178 # number of hidden layer neurons
batch_size = 6 # how many episodes before doing a policy update
learning_rate = 0.009 # How big/small the steps are in the gradient ascent
gamma = 0.99 # discount factor for reward
learning_decay = 50 #Learning rate decay (not used currently)
D = 178 # input dimensionality
total_episodes = 1000 #duhh

In [47]:
def discount_rewards(r):
    # Calculates a reward to reduce rewards gained towards the end of an episode.
    # Not used except for gamma which is used to regulate prioritization of current vs future rewards.
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [48]:
tf.compat.v1.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action 0 or 1.
observations = tf.compat.v1.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.compat.v1.get_variable("W1", shape=[D, H],
           initializer=tf.initializers.GlorotUniform())

layer1 = tf.nn.tanh(tf.matmul(observations,W1))
W11 = tf.compat.v1.get_variable("W11", shape=[H, 89],
           initializer=tf.initializers.GlorotUniform())


layer11 = tf.nn.relu(tf.matmul(layer1, W11))
W2 = tf.compat.v1.get_variable("W2", shape=[89, 1],
           initializer=tf.initializers.GlorotUniform())

score = tf.matmul(layer11,W2)

probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.compat.v1.trainable_variables()
input_y = tf.compat.v1.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.compat.v1.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.compat.v1.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)

newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad1")
W11Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad11")# Placeholders to send the final gradients through when we update.
W2Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W11Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

In [49]:
#stores states, actions, rewards for an episode
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
episode_rewards = []

#lists that maybe used to visualise model training/updates
policy_gradient = []
Q_approx = []
loss_list = []
predictions = []

In [50]:
init = tf.compat.v1.global_variables_initializer()
# Launch the graph
with tf.compat.v1.Session() as sess:
    
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)
        
        
        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            episode_rewards.append(np.sum(drs))
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                policy_gradient.append(gradBuffer)
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W11Grad:gradBuffer[1],W2Grad:gradBuffer[2]})

                loss_list.append(loss)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                
                print('Average reward for episode %f.  Total average reward %f.' % (reward_sum//batch_size, running_reward//batch_size))
                
                reward_sum = 0

            observation = env.reset()
    #get test set with labels. 
    test_set, test_labels = env.validate()
    
    for i in test_set.values:
        # predict on each test row and append to list
        prediction = sess.run(probability, feed_dict={observations:np.reshape(i,[1,D])})
        predictions.append(prediction)
        
            
print(episode_number,'Episodes completed.')


Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 2.000000.  Total average reward -1.000000.
Average reward for episode 1.000000.  Total average reward -1.000000.
Average reward for episode 3.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 1.000000.  Total average reward -1.000000.
Average reward 

In [51]:
pred_clean = []
for i in predictions:
    if i < .50:
        i = 0
        pred_clean.append(i)
    else:
        i = 1
        pred_clean.append(i)

In [52]:
cf_report = classification_report(test_labels,pred_clean)

In [53]:
print(cf_report)

              precision    recall  f1-score   support

           0       0.86      0.32      0.47      1843
           1       0.22      0.78      0.35       457

    accuracy                           0.41      2300
   macro avg       0.54      0.55      0.41      2300
weighted avg       0.73      0.41      0.44      2300



In [54]:
cf = confusion_matrix(test_labels,pred_clean,normalize='all')

In [55]:
cf

array([[0.25913043, 0.54217391],
       [0.04391304, 0.15478261]])